<a href="https://colab.research.google.com/github/LSDtopotools/lsdtt_notebooks/blob/master/lsdtopotools/examples_for_ppl_who_asked/rasterise_bgs_geology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rasterising a BGS geologic map

Written by Simon M Mudd 24/10/2024

## First install gdal

You need the gdal command line tools.

You will also need the osgeo package, but this is already installed on google colab

In [2]:
!apt install gdal-bin &> /dev/null
!gdalinfo --version

GDAL 3.6.4, released 2023/04/17


## Define the functions

Here are the functions we will use.

In [22]:
import os
from os.path import exists
from osgeo import ogr, osr
import csv
from osgeo import gdal
from osgeo.gdalconst import GA_ReadOnly


def get_filename_without_extension(file_name):
    return os.path.splitext(file_name)[0]

def dict_to_csv(data_dict, csv_filename):
    with open(csv_filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['ID', 'rocktype'])
        for key, value in data_dict.items():
            writer.writerow([key, value])


def Rasterize_BGS_geologic_maps(shapefile_name):

    # The shapefile to be rasterized:
    print('Rasterize ' + shapefile_name)
    #get path and filename seperately
    shapefilefilepath = "./"
    shapefilename = shapefile_name
    shapefileshortname = get_filename_without_extension(shapefile_name)

    print("Shapefile name is: "+shapefilename)

    # now get the the fields from the shapefile
    daShapefile = shapefile_name

    dataSource = ogr.Open(daShapefile)
    daLayer = dataSource.GetLayer(0)

    # lets see what the layers are
    print("Let me tell you what the names of the fields are!")
    layerDefinition = daLayer.GetLayerDefn()
    for i in range(layerDefinition.GetFieldCount()):
        print(layerDefinition.GetFieldDefn(i).GetName())


    # The raster file to be created and receive the rasterized shapefile
    outrastername = shapefileshortname + '.tif'
    outraster = shapefilefilepath+os.sep+ outrastername
    outcsv = shapefilefilepath+os.sep+shapefileshortname+'_lithokey.csv'
    print("Full name of out raster is: "+outraster)

    # Rasterize!!
    ## MODIFY THE FOLLOWING LINE TO CHANGE THE GRID SPACING -tr 90 -90
    system_call = 'gdal_rasterize -a BGSREF -l ' + shapefileshortname +' -tr 90 -90 -a_nodata -9999 ' +  shapefile_name + ' ' + outraster
    print("System call is: ")
    print(system_call)
    os.system(system_call)

    # now convert the raster to UTM, as well as delete the stupid TIF
    # The raster file to be created and receive the rasterized shapefile
    outrastername_bil = shapefileshortname + '.bil'
    outraster_bil = shapefilefilepath+os.sep+ outrastername_bil
    print("Full name of out raster is: "+outraster_bil)

    # This assumes UTM zone 30, because why would we do any work in East Anglia?
    system_call2 = 'gdalwarp -t_srs EPSG:32630 -of ENVI -dstnodata -9999 ' +  outraster + ' ' + outraster_bil
    os.system(system_call2)

    # Now get rid of the tif
    system_call3 = 'rm '+ outraster
    os.system(system_call3)


    # Make a key for the bedrock
    geol_dict = dict()
    for feature in daLayer:
        ID = feature.GetField("BGSREF")
        GEOL = feature.GetField("RCS_D")

        if ID not in geol_dict:
            print("I found a new rock type, ID: "+ str(ID)+ " and rock type: " + str(GEOL))
            geol_dict[ID] = GEOL

    print("The rocks are: ")
    print(geol_dict)

    dict_to_csv(geol_dict, outcsv)

    print("All done")

## Import your data and run

**BEFORE YOU RUN THIS** you need to import your shapefile. You need to merge your BGS geologic shapefile and then load all the files (.dbf, .prj, .shp, .shx) into the notebook.

You need to make sure the merged file is  in british national grid.

This will print out an ENVI bil raster with 90m grid spacing.

If you want different grid spacing change the line

`system_call = 'gdal_rasterize -a BGSREF -l ' + shapefileshortname +' -tr 90 -90 -a_nodata -9999 ' +  shapefile_name + ' ' + outraster`

In [23]:
Rasterize_BGS_geologic_maps("merged_bedrock.shp")

Rasterize merged_bedrock.shp
Shapefile name is: merged_bedrock.shp
Let me tell you what the names of the fields are!
LEX_WEB
LEX
LEX_D
LEX_RCS
RCS
RCS_X
RCS_D
RCS_ORIGIN
RANK
BED_EQ_D
MB_EQ_D
FM_EQ_D
SUBGP_EQ_D
GP_EQ_D
SUPGP_EQ_D
MAX_TIME_Y
MIN_TIME_Y
MAX_AGE
MAX_EPOCH
MAX_SUBPER
MAX_PERIOD
MAX_ERA
MAX_EON
BGSTYPE
LEX_RCS_I
LEX_RCS_D
BGSREF
MAP_SRC
MAP_WEB
VERSION
RELEASED
NOM_SCALE
NOM_BGS_YR
UUID
Full name of out raster is: .//merged_bedrock.tif
System call is: 
gdal_rasterize -a BGSREF -l merged_bedrock -tr 90 -90 -a_nodata -9999 merged_bedrock.shp .//merged_bedrock.tif
Full name of out raster is: .//merged_bedrock.bil
I found a new rock type, ID: 262 and rock type: LIMESTONE
I found a new rock type, ID: 140 and rock type: LIMESTONE, SANDSTONE, SILTSTONE AND MUDSTONE
I found a new rock type, ID: 160 and rock type: LIMESTONE
I found a new rock type, ID: 30 and rock type: LIMESTONE, SANDSTONE, SILTSTONE AND MUDSTONE
I found a new rock type, ID: 561 and rock type: SANDSTONE, SILTSTONE 